In [ ]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold,train_test_split
import pickle
from sklearn.metrics import log_loss
#Base models: XGB, LightGBM, CatBoost, ExtraTrees, RandomForest
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier,early_stopping,log_evaluation
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import sklearn
from optuna.integration import CatBoostPruningCallback
import sklearn.metrics

In [6]:
from optuna.integration import CatBoostPruningCallback


In [7]:
#optuna for model tuning
import sklearn.metrics


def xgb_objective(trial, X_train, y_train, X_test, y_test):
    #arams = {
        # 'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        # 'max_depth': trial.suggest_int('max_depth', 3, 10),
        # 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        # 'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.9, 0.1),
        # 'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 0.9, 0.1),
        # 'gamma': trial.suggest_float('gamma', 1e-8, 1.0),
        # 'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        # 'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        # 'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        # 'n_jobs': -1,
        # 'random_state': 42

    #     'lambda': trial.suggest_float('lambda', 1e-3, 10.0),
    #     'alpha': trial.suggest_float('alpha', 1e-3, 10.0),
    #     'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
    #     'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
    #     'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
    #     'n_estimators': 10000,
    #     'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
    #     'random_state': trial.suggest_categorical('random_state', [2020]),
    #     'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    # }
    # clf = XGBClassifier(**params)
    # clf.fit(X_train, y_train)
    # return log_loss(y_test, clf.predict_proba(X_test))
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

def lgb_objective(trial, X_train, y_train, X_test, y_test):
    params = {
        # # 'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        # # 'max_depth': trial.suggest_int('max_depth', 3, 10),
        # # 'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        # # 'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.9, 0.1),
        # # 'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 0.9, 0.1),
        # # 'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        # # 'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        # # 'n_jobs': -1,
        # # 'random_state': 42,
        # "n_estimators": trial.suggest_categorical("n_estimators", [100,1000,10000,25000]),
        # "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        # "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        # "max_depth": trial.suggest_int("max_depth", 3, 12),
        # "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        # "max_bin": trial.suggest_int("max_bin", 200, 300),
        # "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        # "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        # "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        # "bagging_fraction": trial.suggest_float(
        #     "bagging_fraction", 0.2, 0.95, step=0.1
        # ),
        # "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        # "feature_fraction": trial.suggest_float(
        #     "feature_fraction", 0.2, 0.95, step=0.1
        # ),
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "verbose": -1
    }
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_test, label=y_test)
    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    gbm = lgb.train(params, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])

    preds = gbm.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

def cat_objective(trial, X_train, y_train, X_test, y_test):
    #params = {
        # 'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        # 'max_depth': trial.suggest_int('max_depth', 3, 10),
        # 'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        # 'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.9, 0.1),
        # 'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.6, 0.9, 0.1),
        # 'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        # 'random_state': 42,
    #     'loss_function': 'Logloss',
    #     'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 5),
    #     'max_bin': trial.suggest_int('max_bin', 200, 400),
    #     'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
    #     'learning_rate': trial.suggest_uniform('learning_rate', 0.006, 0.018),
    #     'n_estimators':  10000,
    #     'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15]),
    #     'random_state': trial.suggest_categorical('random_state', [2020]),
    #     'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
    #     "verbose": False
    # }
    # clf = CatBoostClassifier(**params)
    # clf.fit(X_train, y_train)
    # return log_loss(y_test, clf.predict_proba(X_test))
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "20gb",
        "eval_metric": "Accuracy",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = cb.CatBoostClassifier(**param)

    pruning_callback = CatBoostPruningCallback(trial, "Accuracy")
    gbm.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test)],
        verbose=0,
        early_stopping_rounds=100,
        callbacks=[pruning_callback],
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    preds = gbm.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)

    return accuracy


def et_objective(trial, X_train, y_train, X_test, y_test):
    n_estimators = trial.suggest_int("n_estimators", 8, 2048)
    max_depth = trial.suggest_int("max_depth", 4, 2048)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 16)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 8)
    criterion = trial.suggest_categorical("criterion", ['gini', 'entropy'])
    
    clf = ExtraTreesClassifier(n_estimators = n_estimators,
                               max_depth = max_depth,
                               min_samples_split = min_samples_split, 
                               min_samples_leaf = min_samples_leaf,
                               criterion = criterion,
                               random_state = 69,
                              )
    
    clf.fit(X_train, y_train)
    return clf.score(X_test, y_test)



In [8]:
def get_train_data():
    # get data
    data = pd.read_csv('../data/train.csv')
    X = data.drop('Label', axis=1)
    y = data['Label']
    return X, y

In [9]:
def get_valid_data():
    # get data
    data = pd.read_csv('../data/validation.csv')
    X = data.drop('Label', axis=1)
    y = data['Label']
    return X, y

In [10]:
from FeatureEngineering import *
def transform_strategy(X):
    X = feabinning(X)
    X = feaInteract(X)
    X = feaCluster(X)
    return X





In [13]:
from typing import List

# lgbm = LGBMClassifier(verbose=-1)
# cat = CatBoostClassifier(verbose=False)
# et = ExtraTreesClassifier()
from sklearn.ensemble import StackingClassifier,BaggingClassifier
from FeatureEngineering import *
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

class EnsembleModel:
    # EnsembleModel for binary classification, using a list of models,output using voting

    def __init__(self):
        xgbc = XGBClassifier()
        lgbm = LGBMClassifier(verbose=-1)
        cat = CatBoostClassifier(verbose=False)
        et = ExtraTreesClassifier()
        rf = RandomForestClassifier()
        svm = SVC(probability=True)
        self.X, self.y = get_train_data()
        self.valid_X, self.valid_y = get_valid_data()
        self.models = [xgbc, lgbm, cat, et,rf]
        self.opt_models = {
            "XGB": (xgbc, xgb_objective),
            "LGBM": (lgbm, lgb_objective),
            "CatBoost": (cat, cat_objective),
            "ExtraTrees": (et, et_objective),
            "RandomForest": (rf, None),
            #"SVM":(svm,None)

        }
        self.hyperparams = {
            "XGB": {},
            "LGBM": {},
            "CatBoost": {},
            "ExtraTrees": {},

        }
        self.n_trials = 50
    
    def feature_engineering(self):
        self.X = transform_strategy(self.X)
        self.valid_X = transform_strategy(self.valid_X)

        
    def bagging_fit(self,X,y,model):
        #base models
        bag_model = BaggingClassifier(estimator=model,n_estimators=10)
        #bag_model.fit(X,y)
        return bag_model

    def stacking_fit(self):
        # Define the base models
        base_models = [
            ('rf', RandomForestClassifier()),
            ('xgb', XGBClassifier()),
            ('lgbm', LGBMClassifier()),
            ('catboost', CatBoostClassifier(verbose=0)),
            ('et', ExtraTreesClassifier()),
            #('svm', SVC(probability=True))

        ]
        # Define the stacking model
        stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())
        stacking_model.fit(self.X, self.y)
        
        print(f" Model: Stacking, log_loss: {log_loss(self.y, stacking_model.predict_proba(self.X))}")
        print(np.mean(stacking_model.predict(self.valid_X) == self.valid_y))

        #multi-level
        # level1 = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())
        # level2 = StackingClassifier(estimators=[('level1', level1)], final_estimator=LogisticRegression())
        # level2.fit(self.X, self.y)
        # print(f" Model: Multi-level Stacking, log_loss: {log_loss(self.y, level2.predict_proba(self.X))}")
        # print(np.mean(level2.predict(self.valid_X) == self.valid_y))

    def bag_and_stack_fit(self):
        base_models = [
            ('rf', RandomForestClassifier()),
            ('xgb', XGBClassifier()),
            ('lgbm', LGBMClassifier()),
            ('catboost', CatBoostClassifier(verbose=0)),
            ('et', ExtraTreesClassifier()),
            #('svm', SVC(probability=True))

        ]
        # Define the stacking model
        bagged_models = [(name,self.bagging_fit(self.X,self.y,model)) for name,model in base_models]
        stacking_model = StackingClassifier(estimators=bagged_models, final_estimator=LogisticRegression(),cv=5,verbose=1)
        #l2_model = StackingClassifier(estimators=[("level1",stacking_model)], final_estimator=LogisticRegression())
        stacking_model.fit(self.X, self.y)
        with open("final_bagged_stacking_model.pkl", "wb") as f:
            pickle.dump(stacking_model, f)
        print(f" Model: B&S, log_loss: {log_loss(self.y, stacking_model.predict_proba(self.X))}")
        valid_acc = np.mean(stacking_model.predict(self.valid_X) == self.valid_y)
        print(valid_acc)
        with open(f"final_bagged_{valid_acc}_stacking_model.pkl", "wb") as f:
            pickle.dump(stacking_model, f)
    def baseline_fit(self):
        
        print("Start baseline training")
        print("===============================")
        for model_name, (model, _) in self.opt_models.items():
            
            model.fit(self.X, self.y)
            print(f" Model: {model_name}, log_loss: {log_loss(self.y, model.predict_proba(self.X))}")

    def validation_test(self):
        print("Start validation test")
        print("===============================")
        print(np.mean(self.predict(self.valid_X) == self.valid_y))
            
        

    def opt_fit(self, X, y):
        print("Start opt training")
        print("===============================")
        # get the best hyperparameters for each model

        train_X, test_X, train_y, test_y = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        for model_name, (model, model_obj) in self.opt_models.items():
            
            print("Now tuning model: ", model)
            optuna.logging.set_verbosity(optuna.logging.INFO)
            study = (
                optuna.create_study(
                    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
                    direction="maximize",
                )
            )

            study.optimize(
                lambda trial: model_obj(trial, train_X, train_y, test_X, test_y),
                n_trials=self.n_trials,
            )
            params = study.best_params
            self.hyperparams[model_name] = params

        # set the best hyperparameters for each model
        for model_name, (model, _) in self.opt_models.items():
            model.set_params(**self.hyperparams[model_name])
        #conduct k-fold
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            for model_name, (model, _) in self.opt_models.items():
                model.fit(X_train, y_train)
                
        
        #full data training
        # for model_name, (model, _) in self.opt_models.items():
        #     model.fit(X, y)
        #     print(f" Model: {model_name}, log_loss: {log_loss(y, model.predict_proba(X))}")

    def predict(self,X):
        predictions = []
        for model in self.models:
            predictions.append(model.predict(X))
        return self._voting(predictions)

    def _voting(self, predictions):
        # voting for binary classification
        votes = [0] * len(predictions[0])
        for prediction in predictions:
            for i in range(len(prediction)):
                if prediction[i] == 1:
                    votes[i] += 1
        return [1 if vote > len(predictions) / 2 else 0 for vote in votes]

In [14]:
for i in range(10):
    new_ens = EnsembleModel()
    new_ens.baseline_fit()
    new_ens.validation_test()
    new_ens.bag_and_stack_fit()




Start baseline training
 Model: XGB, log_loss: 0.03491625373577679
 Model: LGBM, log_loss: 0.07916571232377027
 Model: CatBoost, log_loss: 0.13417159078186291
 Model: ExtraTrees, log_loss: 2.2204460492503136e-16
 Model: RandomForest, log_loss: 0.09070419632586077
Start validation test
0.9075
[LightGBM] [Info] Number of positive: 1600, number of negative: 1600
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000253 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 3200, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.492812 -> initscore=-0.028752
[LightGBM] [Info] Start training from score -0.028752
[LightGBM] [Info] Number of positive: 1600, number of negative: 1600
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000158 seconds.
You can set `force_col_wise=true` to remove 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    7.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LightGBM] [Info] Number of positive: 1280, number of negative: 1280
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000168 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 2560, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491797 -> initscore=-0.032815
[LightGBM] [Info] Start training from score -0.032815
[LightGBM] [Info] Number of positive: 1280, number of negative: 1280
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 2560, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502344 -> initscore=0.009375
[LightGBM] [Info] Start training from score 0.009375
[LightGBM] [Info] Numb

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   45.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.1s finished


 Model: B&S, log_loss: 0.05993363375364951
0.91
Start baseline training
 Model: XGB, log_loss: 0.03491625373577679
 Model: LGBM, log_loss: 0.07916571232377027
 Model: CatBoost, log_loss: 0.13417159078186291
 Model: ExtraTrees, log_loss: 2.2204460492503136e-16
 Model: RandomForest, log_loss: 0.09145846712257692
Start validation test
0.9075
[LightGBM] [Info] Number of positive: 1600, number of negative: 1600
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000199 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 3200, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.489063 -> initscore=-0.043757
[LightGBM] [Info] Start training from score -0.043757
[LightGBM] [Info] Number of positive: 1600, number of negative: 1600
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000156 secon

...

 Model: B&S, log_loss: 0.05357828140860569
0.9125
Start baseline training
 Model: XGB, log_loss: 0.03491625373577679
 Model: LGBM, log_loss: 0.07916571232377027
 Model: CatBoost, log_loss: 0.13417159078186291
 Model: ExtraTrees, log_loss: 2.2204460492503136e-16
 Model: RandomForest, log_loss: 0.09099263597058344
Start validation test
0.9125
[LightGBM] [Info] Number of positive: 1600, number of negative: 1600
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 3200, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.484375 -> initscore=-0.062520
[LightGBM] [Info] Start training from score -0.062520
[LightGBM] [Info] Number of positive: 1600, number of negative: 1600
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000135 sec

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    7.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LightGBM] [Info] Number of positive: 1280, number of negative: 1280
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000134 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 2560, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.520312 -> initscore=0.081295
[LightGBM] [Info] Start training from score 0.081295
[LightGBM] [Info] Number of positive: 1280, number of negative: 1280
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000131 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 2560, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.494531 -> initscore=-0.021876
[LightGBM] [Info] Start training from score -0.021876
[LightGBM] [Info] Numb

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    7.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   45.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.1s finished


 Model: B&S, log_loss: 0.054247150865997164
0.915


In [ ]:
def get_train_data():
    # get data
    data = pd.read_csv('../data/train.csv')
    X = data.drop('Label', axis=1)
    y = data['Label']
    return X, y

In [ ]:
X,y = get_train_data()

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=UserWarning)
# ensem = EnsembleModel()
# ensem.opt_fit(X, y)
ensem = EnsembleModel()
ensem.baseline_fit(X, y)



In [ ]:
np.mean(ensem.predict(valid_X) == valid_y)

In [ ]:
np.mean(ensem.predict(X) == y)

In [ ]:
valid = pd.read_csv('../data/validation.csv')

In [ ]:
valid_X = valid.drop('Label', axis=1)
valid_y = valid['Label']